In [11]:
#This script takes a "Guide sequence" generated by CRISPR Design Tool (http://tools.genome-engineering.org)
#and generates the oligos needed for their cloning into a pX-series vector.
#For more info check: Ran, F. A. et al. Genome engineering using the CRISPR-Cas9 system. Nat Protoc 8, 2281–2308 (2013).

#Input variable: sequence_list - It's a list of tuples, [('seq_name','sequence'),...].
#Each tuple contains the name of the sequence and the sequence.

#author: alessio.marcozzi@gmail.com
#version: 1.4
#date last revision: January 2016


##Insert your input sequence(s) here and hit 'Play'.
##Do not include the PAM tor your input sequence(s)!
sequence_list = [('OCA2_8','AATAAAAAGTAAGTCCAGGC'),
                 ('OCA2_1','CACTGAAGCCTCGACTTTCC'),
                 ('OCA2_2','AGTCGAGGCTTCAGTGAGTC'),
                 ('HERC2_9','ATATATAAAATTTAATCAAC'),
                 ('HERC2_1','GGTGGGAGGTTCAGATTGAA'),
                 ('RE_Ctrl','AGTCTGTCTAGACTCGAAAA'),
                 ('HERC2_10','TGATTAAATTTTATATATGG')]


##Global variable
BAD_SEQUENCES = []


##Functions declaration
def check(sequence):
    '''Checks a DNA sequence.
    Returns False if somethin inappropriate is found.'''
    global BAD_SEQUENCES
    #check for non ATCG letters
    for letter in sequence:
        if letter.upper() not in 'ATCG':
            print("\n>>> Oligos design for {} FAILED !!!".format(guide_name))
            print("##Warning!! Non ATCG base found! in {}".format((guide_name,sequence)))
            BAD_SEQUENCES.append((guide_name,sequence))
            return False
    #check for the presence of unwanted restriction sites
    re_sites = {'BbsI_F':'GAAGAC',
                'BbsI_R':'GTCTTC',
                'XbaI':'TCTAGA',
                'KpnI':'GGTACC'}
    for re in re_sites:
        if re_sites[re] in sequence:
            print("\n>>> Oligos design for {} FAILED !!!".format(guide_name))
            print("##Warning!! {} restriction site found! in {}".format(re,(guide_name,sequence)))
            BAD_SEQUENCES.append((guide_name,sequence))
            return False
    return True      

def reverse(sequence):
    '''Takes a DNA sequence as input.
    Returns its reverse sequence.'''
    reversed_sequence = ''
    for n in range(len(sequence)):
        reversed_sequence += sequence[-(1+n)]
    return reversed_sequence

def complement(sequence):
    '''Takes a DNA sequence as input.
    Returns its complementary sequence.'''
    complementary_sequence = ''
    for base in sequence:
        if base == 'A':
            complementary_sequence += 'T'
        elif base == 'T':
            complementary_sequence += 'A'
        elif base == 'C':
            complementary_sequence += 'G'
        elif base == 'G':
            complementary_sequence += 'C'
        else:
            raise ValueError("##Error!! Non ATCG base found! in {}".format(sequence))
    return complementary_sequence

def output(sequence, guide_name):
    '''Prints a human readable output.'''
    forward_extra = 'CACCG' # 5'-CACCG-3' BbsI + G
    reverse_extra = 'CAAA' # 3'-CAAA-5' BbsI
    fd = forward_extra + sequence
    rv = complement('G' + sequence) + reverse_extra
    print("\n>>> Oligos design for {}\n".format(guide_name))
    print("Annealing view")
    print("5'-", fd, "-3'")
    print("       ", "*||||||||||||||||||||") # '*' indicates the extra G "An additional cytosine
                                              # As specified in Genome engineering using the CRISPR-Cas9 system.
                                              # Nature Protocols, 8(11), 2281–2308.
                                              # http://doi.org/10.1038/nprot.2013.143
                                              # (‘C’ in gray rectangle) is appended in the reverse primer
                                              # directly 3′ to the target sequence to allow guanine
                                              # as the first base of the U6 transcript.
    print("   ", "3'-", rv, "-5'")
    print("\nOligos to order (5'-3' orientation)")
    print("{}_Top    : {}".format(guide_name,fd))
    print("{}_Bottom : {}\n".format(guide_name,reverse(rv)))

#print('Done')


##Body
for item in sequence_list:
    guide_name = item[0]
    sequence = item[1]
    if check(sequence) == True:
        output(sequence, guide_name)
    if len(sequence) > 20:
        print('Sequence {}:{} is longer than expected'.format(sequence,guide_name))
        BAD_SEQUENCES.append((guide_name,sequence))
        

##Errors tracking
if len(BAD_SEQUENCES) > 0:
    print('#################################################')
    print("##WARNING! One or more sequences are incorrect.##")
    print('#################################################')
    print("Please check the following sequence(s):")
    print(BAD_SEQUENCES)
else:
    print("All the guides were correctly designed!")


>>> Oligos design for OCA2_8

Annealing view
5'- CACCGAATAAAAAGTAAGTCCAGGC -3'
        *||||||||||||||||||||
    3'- CTTATTTTTCATTCAGGTCCGCAAA -5'

Oligos to order (5'-3' orientation)
OCA2_8_Top    : CACCGAATAAAAAGTAAGTCCAGGC
OCA2_8_Bottom : AAACGCCTGGACTTACTTTTTATTC


>>> Oligos design for OCA2_1

Annealing view
5'- CACCGCACTGAAGCCTCGACTTTCC -3'
        *||||||||||||||||||||
    3'- CGTGACTTCGGAGCTGAAAGGCAAA -5'

Oligos to order (5'-3' orientation)
OCA2_1_Top    : CACCGCACTGAAGCCTCGACTTTCC
OCA2_1_Bottom : AAACGGAAAGTCGAGGCTTCAGTGC


>>> Oligos design for OCA2_2

Annealing view
5'- CACCGAGTCGAGGCTTCAGTGAGTC -3'
        *||||||||||||||||||||
    3'- CTCAGCTCCGAAGTCACTCAGCAAA -5'

Oligos to order (5'-3' orientation)
OCA2_2_Top    : CACCGAGTCGAGGCTTCAGTGAGTC
OCA2_2_Bottom : AAACGACTCACTGAAGCCTCGACTC


>>> Oligos design for HERC2_9

Annealing view
5'- CACCGATATATAAAATTTAATCAAC -3'
        *||||||||||||||||||||
    3'- CTATATATTTTAAATTAGTTGCAAA -5'

Oligos to order (5'-3' orientation)
HERC